 ### Target Offload

 In the OpenMP 5.0 implementation the  `OMP_TARGET_OFFLOAD`  environment variable was defined to change  _default_  offload behavior.  By  _default_  the target  (region) is executed on the host if the target device  does not exist or the implementation does not support the target device.   
Last sentence uses words of the 5.0 spec pg. 21 lines 7-8

 In an OpenMP 5.0 compliant implementation, setting the   `OMP_TARGET_OFFLOAD`  variable to  `MANDATORY`  will  force the program to terminate execution when a  `target`   construct is encountered and the target device is not supported or is not available. With a value  `DEFAULT`  the target region will execute on a device if the  device exists and is supported by the implementation, otherwise it will execute on the host. Support for the  `DISABLED`  value is optional; when it is supported the behavior is as if only the  host device exists (other devices are considered non-existent to the runtime),  and target regions are executed on the host.  

 The following example reports execution behavior for different  values of the  `OMP_TARGET_OFFLOAD`  variable. A handy routine  for extracting the  `OMP_TARGET_OFFLOAD`  environment variable value is deployed here, because the OpenMP API does not have a routine  for obtaining the value. 
({yet}).

 Note:  The example issues a warning when a pre-5.0 implementation is used, indicating that the  `OMP_TARGET_OFFLOAD`  is ignored. The value of the  `OMP_TARGET_OFFLOAD`  variable is reported  when the  `OMP_DISPLAY_ENV`   environment variable is set to  `TRUE`  or  `5.0.1BOSE` .

 
cexample[5.0]{target_offload_control}{1}

 
ffreeexample[5.0]{target_offload_control}{1}